In [ ]:
import pandas as pd
import os
import glacierml as gl
from tqdm import tqdm
import matplotlib.pyplot as plt
import path_manager as pm
[
        home_path, data_path, RGI_path, glathida_path, 
        ref_path, coregistration_testing_path, 
        arch_test_path, LOO_path
] = pm.set_paths()

In [ ]:
df = gl.coregister_data(data_path,'4')
df = df.reset_index().drop('index', axis = 1)

dft = df.copy()

trfeat, tefeat, trlabs, telabs = gl.split_data(df)

In [ ]:
results = pd.DataFrame(
    columns = ['Arch','Metric','Loss']
)

In [ ]:
for loss_metric in os.listdir(arch_test_path):
    loss = []
    architectures = []
    params = []
    for arch in tqdm(os.listdir(os.path.join(arch_test_path,loss_metric))):
        architectures.append(arch)

        model_path = os.path.join(arch_test_path,loss_metric,arch)
        model = gl.load_dnn_model(model_path)
        
        score = model.evaluate(tefeat,telabs,verbose = 0)
        loss.append(score)
        
        num_params = model.count_params()
        params.append(num_params)
        
    loss = pd.Series(loss,name = 'Loss')
    params = pd.Series(params,name = 'Parameters')
    architectures = pd.Series(architectures,name = 'Arch')
    metric = pd.Series(loss_metric,name = 'Metric',index = loss.index)
    res_temp = pd.concat([architectures,metric,loss,params/273],axis = 1)
    results = pd.concat([results,res_temp],axis = 0)

In [ ]:
results.to_pickle(os.path.join(arch_test_path,'results.pkl'))

In [ ]:
results = pd.read_pickle(os.path.join(arch_test_path,'results.pkl'))

In [ ]:
mae = results[results['Metric'] == 'mae']
mse = results[results['Metric'] == 'mse']

In [ ]:
mse['MSE_Rank'] = mse['Loss'].rank()
mae['MAE_Rank'] = mae['Loss'].rank()

df_ranked = pd.merge(mse[
    ['Arch', 'MSE_Rank','Parameters']
], mae[
    ['Arch', 'MAE_Rank','Parameters']
], on=['Arch','Parameters'])

df_ranked['Rank_Difference'] = abs(df_ranked['MSE_Rank'] - df_ranked['MAE_Rank'])
min_difference = df_ranked['Rank_Difference'].min()

threshold = 5  

min_difference_architectures = df_ranked[
    df_ranked['Rank_Difference'] <= threshold][
    ['Arch', 'Rank_Difference','Parameters']
].sort_values('Rank_Difference')

print(f'Architectures with a rank difference less than or equal to {threshold}:')
print(min_difference_architectures)